In [1]:
# LSTM
import pandas as pd
import numpy as np
from dataset import LABELS
from sklearn.preprocessing import LabelEncoder
import random


c:\Users\mikol\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from dataset import silence_detection_training, silence_detection_validation, silence_detection_full

In [3]:
from models import Lstm

In [61]:
model = Lstm(lstm_units=64, dropout_rate=0.2, epoch=4, batch_size=32, learning_rate=0.001, input_shape=(39,44), num_classes=2, model_path='models\\lstm_silence.h5')

In [82]:
model.train(silence_detection_full)

Epoch 1/4
435/435 [==============================] - ETA: 0s - loss: 0.0702 - accuracy: 0.9887
Epoch 1: accuracy improved from -inf to 0.98872, saving model to models\lstm_silence.h5
435/435 [==============================] - 98s 225ms/step - loss: 0.0702 - accuracy: 0.9887
Epoch 2/4
435/435 [==============================] - ETA: 0s - loss: 0.0198 - accuracy: 0.9966
Epoch 2: accuracy improved from 0.98872 to 0.99655, saving model to models\lstm_silence.h5
435/435 [==============================] - 65s 149ms/step - loss: 0.0198 - accuracy: 0.9966
Epoch 3/4
435/435 [==============================] - ETA: 0s - loss: 0.0143 - accuracy: 0.9974
Epoch 3: accuracy improved from 0.99655 to 0.99741, saving model to models\lstm_silence.h5
435/435 [==============================] - 69s 159ms/step - loss: 0.0143 - accuracy: 0.9974
Epoch 4/4
435/435 [==============================] - ETA: 0s - loss: 0.0128 - accuracy: 0.9973
Epoch 4: accuracy did not improve from 0.99741
435/435 [==================

In [60]:
model.predict(silence_detection_validation)

688/688 [==============================] - 25s 32ms/step


array([23,  1,  1, ..., 23,  1,  1], dtype=int64)

In [ ]:
from dataset import TensorflowDataset

# final model class

In [38]:
import tensorflow as tf
import numpy as np
from models import Lstm, Gru, Transformer
from dataset import LABELS
import pandas as pd

class final_model:
    def __init__(self, model) -> None:

        assert model in [Lstm, Gru, Transformer]
        # self.model = model(model_path = f'models\\model_{model.__name__.lower()}_final_version.h5')
        self.model = model(model_path = f'models\\transformer_mod.h5')
        self.name = model.__name__.lower()

        test = pd.read_pickle('extracted_features\\features_test.pkl')
        dataset = tf.data.Dataset.from_tensor_slices((test, np.zeros(len(test))))
        self.test = dataset

        test_silence = pd.read_pickle('extracted_features\\features_test_silence.pkl')
        dataset = tf.data.Dataset.from_tensor_slices((test_silence, np.zeros(len(test_silence))))
        self.test_silence = dataset

        LABELS_dict = {i:LABELS[i] for i in range(len(LABELS))}
        LABELS_dict[10] = 'unknown'
        self.LABELS_dict = LABELS_dict

        self.predicted_labels = None
        self.silence_labels = None

    def predict_silence(self):
        model_sil = tf.keras.models.load_model('models\\lstm_silence.h5')
        self.silence_labels = np.argmax(model_sil.predict(self.test.batch(32)), axis=1)
    
    def predict_label(self):
        self.predicted_labels = self.model.predict(self.test)

    def save_results(self):
        x = pd.concat([pd.DataFrame(self.predicted_labels), pd.DataFrame(self.silence_labels)], axis=1)
        x.columns = ['label', 'silence']
        res = np.where(x['silence'] == 0 , 'silence', x['label'].apply(lambda x: self.LABELS_dict[x]))
        res = pd.DataFrame(res)
        res.columns = ['label_prediction']
        res.to_csv(f'results\\submission_{self.name}.csv', index=False, header=True)

In [40]:
d = final_model(Transformer)

In [41]:
d.predict_silence()
d.predict_label()

15854/15854 [==============================] - 75s 5ms/step


In [50]:
x = pd.concat([pd.DataFrame(d.predicted_labels), pd.DataFrame(d.silence_labels)], axis=1)
x.columns = ['label', 'silence']
res = np.where(x['silence'] == 0 , 'silence', tr(x))
res = pd.DataFrame(res)
res.columns = ['label_prediction']

In [53]:
x.label.value_counts()

15    158406
24       113
23        19
Name: label, dtype: int64

In [49]:
def tr(x):
    try:
        x['label'].apply(lambda x: d.LABELS_dict[x])
    except:
        'unknown'

In [46]:
pd.DataFrame(d.predicted_labels).value_counts()

15    158406
24       113
23        19
dtype: int64

In [33]:
x = pd.concat([pd.DataFrame(d.predicted_labels), pd.DataFrame(d.silence_labels)], axis=1)
x.columns = ['label', 'silence']


In [34]:
x

,label,silence
0,22,1
1,21,1
2,16,1
3,16,1
4,19,1
...,...,...
158533,21,1
158534,22,1
158535,22,1
158536,22,1


In [27]:
d.save_results()

In [28]:
pd.read_csv('results\\submission_lstm.csv').value_counts()

label_prediction
down                3226
on                   621
unknown              192
no                   187
silence               84
left                   2
dtype: int64

In [65]:
from dataset import LABELS

In [72]:
LABELS_dict = {i:LABELS[i] for i in range(len(LABELS))}
LABELS_dict[10] = 'unknown'

In [73]:
LABELS_dict

{0: 'yes',
 1: 'no',
 2: 'up',
 3: 'down',
 4: 'left',
 5: 'right',
 6: 'on',
 7: 'off',
 8: 'stop',
 9: 'go',
 10: 'unknown'}

In [74]:
x = pd.concat([pd.DataFrame(d.predicted_labels), pd.DataFrame(d.predicted_labels)], axis=1)
x.columns = ['label', 'silence']
x = x[x.label<11]
res = np.where(x['silence'] == 1 , 'silence', x['label'].apply(lambda x: LABELS_dict[x]))

In [81]:
pd.DataFrame(res).to_csv('submission.csv', index=False, header=True)

In [30]:
d.predict_label()

 1390/15854 [=>............................] - ETA: 6:42

KeyboardInterrupt: 

In [9]:
test = pd.read_pickle('extracted_features\\features_test.pkl')

In [16]:
test_silence = pd.read_pickle('extracted_features\\features_test_silence.pkl')

UnpicklingError: pickle data was truncated

In [17]:
import tensorflow as tf
import numpy as np

dataset = tf.data.Dataset.from_tensor_slices((test, np.zeros(len(test))))

In [18]:
from models import Lstm, Gru, Transformer


In [19]:
model = Lstm()

In [20]:
model.predict(dataset)

  432/15854 [..............................] - ETA: 5:41

KeyboardInterrupt: 

In [12]:
model = tf.keras.models.load_model('models\\lstm_silence.h5')

In [14]:
preds = model.predict(dataset)

1068/4955 [=====>........................] - ETA: 1:59

KeyboardInterrupt: 

In [17]:
# LSTM
import pandas as pd
import numpy as np
from dataset import LABELS
from sklearn.preprocessing import LabelEncoder
import random


In [18]:
len(train)

62923

In [19]:
len(val)

6879

In [29]:
from sklearn.utils import resample
train = pd.read_pickle('extracted_features\\silence_detection_training.pkl')
val = pd.read_pickle('extracted_features\\silence_detection_validation.pkl')

# Separate 'silence' and 'non-silence' observations
train_silence = [x for x in train if x[1] == 'silence']
train_non_silence = [x for x in train if x[1] != 'silence']

# Resample 'silence' observations to match the number of 'non-silence' observations
train_silence_resampled = resample(train_silence, replace=True, n_samples=len(train_non_silence))

# Combine resampled 'silence' and 'non-silence' observations into the final training set
train_final = train_non_silence + train_silence_resampled

# Change all 'non-silence' labels greater than 'silence' to 'non-silence'
for i in range(len(train_final)):
    if train_final[i][1] != 'silence':
        train_final[i][1] = 'non-silence'

for i in range(len(val)):
    if val[i][1] != 'silence':
        val[i][1] = 'non-silence'

# randomly sort the training set
random.shuffle(train_final)


In [30]:
X_train = np.array([x[0] for x in train_final])
y_train = np.array([x[1] for x in train_final])
X_val = np.array([x[0] for x in val])
y_val = np.array([x[1] for x in val])

In [31]:


# Create a label encoder object
label_encoder = LabelEncoder()

# Fit the label encoder using your labels (combine both train and val labels if they have different classes)
label_encoder.fit(np.concatenate((y_train, y_val)))

# Transform your string labels to integer labels for both training and validation sets
y_train_encoded = label_encoder.transform(y_train)
y_val_encoded = label_encoder.transform(y_val)

In [32]:
(y_train!='silence').sum()

57923

In [33]:
(y_train=='silence').sum()

57923

In [27]:
from dataset import silence_detection_validation

In [28]:
silence_detection_validation

<ShuffleDataset element_spec=(TensorSpec(shape=(39, 44), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.int32, name=None))>

In [37]:
len(np.unique(y_train_encoded))

2

In [38]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding, Dropout
from tensorflow.keras.layers import LSTM, Dropout, Dense, Bidirectional, TimeDistributed, BatchNormalization

results = pd.DataFrame(columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])

# Set your model's hyperparameters
for lstm_units in [64]:
    for dropout_rate in [0.2]:
            for epoch in [4]:
                  for batch in [32]:

                        num_classes = len(np.unique(y_train_encoded))  # Number of unique classes in your dataset

                        input_shape = (39, 44)

                        model = Sequential([
                            Bidirectional(LSTM(lstm_units, return_sequences=True), input_shape=input_shape),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Bidirectional(LSTM(lstm_units, return_sequences=True)),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            LSTM(lstm_units, return_sequences=True),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(64, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            TimeDistributed(Dense(32, activation='relu')),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            LSTM(lstm_units),
                            BatchNormalization(),
                            Dropout(dropout_rate),
                            Dense(num_classes, activation='softmax')
                        ])


                        # Compile the model
                        model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

                        # Train the model with your training set and validate it with your validation set
                        epochs = epoch
                        batch_size = batch
                        history = model.fit(X_train, y_train_encoded, epochs=epochs, batch_size=batch_size, validation_data=(X_val, y_val_encoded))

                        results = np.concatenate((results, pd.DataFrame([[lstm_units, dropout_rate, epoch, batch, history.history['loss'][-1], history.history['loss'], history.history['accuracy'][-1], history.history['accuracy'], history.history['val_loss'][-1], history.history['val_loss'], history.history['val_accuracy'][-1], history.history['val_accuracy']]], 
                                                                        columns=['lstm_units', 'dropout_rate', 'epoch', 'batch', 'loss', 'loss_max', 'accuracy', 'accuracy_max', 'val_loss', 'val_loss_max', 'val_accuracy', 'val_accuracy_max'])), axis=0)


Epoch 1/4
3621/3621 [==============================] - 309s 77ms/step - loss: 0.0224 - accuracy: 0.9942 - val_loss: 0.0151 - val_accuracy: 0.9956
Epoch 2/4
3621/3621 [==============================] - 229s 63ms/step - loss: 0.0111 - accuracy: 0.9974 - val_loss: 0.0099 - val_accuracy: 0.9980
Epoch 3/4
3621/3621 [==============================] - 226s 62ms/step - loss: 0.0070 - accuracy: 0.9983 - val_loss: 0.0131 - val_accuracy: 0.9983
Epoch 4/4
3621/3621 [==============================] - 221s 61ms/step - loss: 0.0055 - accuracy: 0.9987 - val_loss: 0.0110 - val_accuracy: 0.9974


In [39]:
# save model 
model.save('models\\lstm_silence_model.h5')

In [40]:
# upload model from models\\
from tensorflow.keras.models import load_model
model = load_model('models\\lstm_silence_model.h5')

In [41]:
model